## Web Scraping: Fortune Global 500 CEOs (2019)

In [ ]:
import awscli
import sys
import selenium
import unittest
import boto3
import pandas as pd
import tabulate
import time
import requests
import csv
import socket 
import numpy as np
import dataframe
from bs4 import BeautifulSoup
from tabulate import tabulate
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [ ]:
# For 1st part (~266)
browser = webdriver.Chrome(r"C:\Users\...\chromedriver.exe")
browser.get('https://fortune.com/global500/2019/walmart')

#For the  1st part

df1 = pd.DataFrame(columns=['CEO','Sector', 'Industry', 'HQ Location', 'Website', 'Years on Global 500 List', 'Employees'])
c1 = []

for i in [x for x in range(266)]:         
    table = browser.find_element_by_css_selector('tbody')
    
    if i > 266:
        break


    try:
        print("Scraping Page no. " + str(i))
        i = i + 1
        values1 =[]
    
        for row in table.find_elements_by_css_selector('tr'):
            value1 = str(([cell.text for cell in row.find_elements_by_css_selector('td.dataTable__value--3n5tL.dataTable__valueAlignLeft--3uvNx')])).strip('[]').strip("''")
            values1.append(value1)
            comp1 = str(([c.text for c in row.find_elements_by_xpath('//*[@id="content"]/div[1]/div[1]/div/h1/span')])).strip('[]').strip("''")    
        c1.append(comp1)
        
        s1 = pd.Series(values1,index=df1.columns)
        cs1 = pd.Series(c1) 
        df1 = df1.append(s1,ignore_index=True)
        
        try:

            WebDriverWait(browser, 6).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.singlePagination__next2--3D89W"))).click()
            time.sleep(5)

        except (socket.gaierror, requests.ConnectionError) as e:
            if e.errno != 10054:
                continue
            reconnect()



    except Exception as e:
        print(e)
        break

df1['company'] = cs1
df1['year']=2019

re_index = pd.Series(range(1,267))  
df1 = df1.set_index([re_index])
print(df1)

browser.quit()


In [ ]:
df1.to_csv(r'C:\Users\...\fg500_CEOs_2019_p1(Apr_25_2020).csv')


In [ ]:
# For 2nd part (268~)
browser = webdriver.Chrome(r"C:\Users\...\chromedriver.exe")
browser.get('https://fortune.com/global500/2019/new-york-life-insurance/')

df2 = pd.DataFrame(columns=['CEO','Sector', 'Industry', 'HQ Location', 'Website', 'Years on Global 500 List', 'Employees'])
c2 = []

for i in [x for x in range(233)]:       
    table = browser.find_element_by_css_selector('tbody')
    
    if i > 233:
        break


    try:
        print("Scraping Page no. " + str(i))
        i = i + 1
        values2 =[]
    
        for row in table.find_elements_by_css_selector('tr'):
            value2 = str(([cell.text for cell in row.find_elements_by_css_selector('td.dataTable__value--3n5tL.dataTable__valueAlignLeft--3uvNx')])).strip('[]').strip("''")
            values2.append(value2)
            comp2 = str(([c.text for c in row.find_elements_by_xpath('//*[@id="content"]/div[1]/div[1]/div/h1/span')])).strip('[]').strip("''")    
        c2.append(comp2)
        
        s2 = pd.Series(values2,index=df2.columns)
        cs2 = pd.Series(c2) 
        df2 = df2.append(s2,ignore_index=True)
        
        try:

            WebDriverWait(browser, 6).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.singlePagination__next2--3D89W"))).click()
            time.sleep(5)

        except (socket.gaierror, requests.ConnectionError) as e:
            if e.errno != 10054:
                continue
            reconnect()



    except Exception as e:
        print(e)
        break

df2['company'] = cs2
df2['year']=2019

re_index1 = pd.Series(range(268,501))  
df2 = df2.set_index([re_index1])

print(df2)

browser.quit()


In [ ]:
df2.to_csv(r'C:\Users\...\fg500_CEOs_2019_p2(Apr_25_2020).csv')

In [ ]:
df_ceo_com=df1.append(df2, ignore_index=False)
print(df_ceo_com)

In [ ]:
df_ceo_com.to_csv(r'C:\Users\...\fg500_CEOs_2019_All(Apr_25_2020).csv')

In [ ]:
from botocore.client import Config
from io import StringIO

ACCESS_KEY_ID = 'your_access_key_id'
ACCESS_SECRET_KEY = 'your_access_secret_key'
BUCKET_NAME = 'ceo-culture-sagemaker'

csv_buffer =StringIO()
df1.to_csv(csv_buffer)

s3 = boto3.resource(
        's3',
        aws_access_key_id = ACCESS_KEY_ID,
        aws_secret_access_key=ACCESS_SECRET_KEY,
        config=Config(signature_version='s3v4')
        )
s3.Bucket(BUCKET_NAME).put_object(Key='fg500_CEOs_2019_All(Apr_25_2020).csv', Body = csv_buffer.getvalue())

print("Uploaded")